<p style="text-align:center;font-family: Arial; font-size:3.0em; font-style:bold"><br>
Silent Disco Glamping Network
</p><br>
<p style="text-align:center;font-family: Arial; font-size:2.0em; font-style:bold"><br>
Le Minh Bien
</p><br>
<p style="text-align:center; font-family: Arial; font-size:1.00em;color:white; font-style:bold">Originally created for: </p>
<p style="text-align:center; font-family: Arial; font-size:1.00em;color:green; font-style:bold">
Project 2 of Mathematical Optimisation: Theory and Application, AMSI Summer School 2026</p>

## Problem formulation
Given a set of potential locations $\mathcal{L}$ (set of airstrips in Australia), $\mathcal{L} = {1, 2, ..., m}$.

Each element $i \in \mathcal{L}$ corresponds to one location point in $\mathbb{R}^2$, where each point is indexed by $l_i = (x_i, y_i) \in \mathbb{R}^2$.

#### Parameters:
* $d_{ij} = ||l_i-l_j||_2$: Euclidean distance between location $i$ and $j$ $(d_{ij} = 0)$
* $w_i >0$: weighted cost factor of location $i$
* $a_i$: maximum number of tents at location $i$
* $k$: number of sites to select
* $\delta$: minimum distance between any two selected sites
* $N$: total number of tents across all sites

#### Decision variable
* $x_i \in \{0,1\}$:
$$
x_i = 
\begin{cases}
1, \text{ if location} i \text{is selected,} \\
0, \text{ otherwise}
\end{cases}
$$

#### Mathematical formulation
$$
\begin{aligned}
\max_{x}\quad 
& \frac{\sum_{i,j \in \mathcal{L}} d_{ij} x_i x_j}{\sum_{i \in \mathcal{L}} w_i x_i} \\[6pt]
\text{s.t.}\quad 
& \sum_{i \in \mathcal{L}} x_i = k \\[4pt]
& x_i + x_j \le 1 \quad \forall (i,j): d_{ij} < \delta \\[4pt]
& \sum_{i \in \mathcal{L}} a_i x_i \ge N \\[4pt]
& x_i \in \{0,1\}
\end{aligned}
$$

Let t = $\sum_{i=1}^mw_ix_i (t>0)$, hence, t is a continuous variable

Then the objective becomes $\max_x \frac{\sum_{i,j \in \mathcal{L}}^md_{ij}x_ix_j}{t}$

Let $f(x) = \sum_{i,j \in \mathcal{L}}^md_{ij}x_ix_j$, the problem becomes $max_x \frac{f(x)}{t}$

Since $f(x)$ is quadratic, $f(\frac{x}{t}) = \frac{1}{t^2}f(x)$
Therefore, maximising $\frac{f(x)}{t}$ is equivalent to maximising $g(x,t) = tf(\frac{x}{t})$

From the Max-sum Diversity Problem, Spiers, Bui and Loxton (2023) have proved that $f$ is concave over the feasible set $K = \{x|\sum_{x=1}^m = k\}$. Since the additional constraints define a subset of $K$, concavity of $f$ is preserved on the feasible region of our problem.

Since $f$ is proved to be concave, $g(x,t)$ is concave as a perspective mapping of function $f$.

Now, our problem is proved to be concave, we can solve it using Outer Approximation.

In [2]:
# Import necessary libraries
import numpy as np
import gurobipy as gp
from gurobipy import Model, GRB
import math
from typing import List, Tuple
from Linearisationtechnique import qp_model, second_linear
import pandas as pd
import plotly.express as px
from sklearn.cluster import KMeans
import plotly
from numba import njit
import plotly.graph_objects as go
from termcolor import colored
import warnings
from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning
from scipy.spatial.distance import cdist

warnings.simplefilter("ignore", category=NumbaDeprecationWarning)
warnings.simplefilter("ignore", category=NumbaPendingDeprecationWarning)
# plotly.graph_objs._scattergl.Scattergl

Create functions to generate distance matrix from locations data

In [3]:
@njit
def haversine(u: np.ndarray, v: np.ndarray) -> float:
    """
    Compute Haversine distance (in km) between two latitude-longitude points.
    """
    R = 6371.0  # Earth radius in km

    lat1, lon1 = np.radians(u)
    lat2, lon2 = np.radians(v)

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = np.sin(dlat / 2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0)**2
    c = 2.0 * np.arcsin(np.sqrt(a))

    return R * c

@njit
def haversine_matrix(locations: np.ndarray) -> np.ndarray:
    """
    Computes a symmetric Haversine distance matrix in km.
    """
    n = locations.shape[0]
    D = np.zeros((n, n))

    for i in range(n):
        for j in range(i + 1, n):
            d = haversine(locations[i], locations[j])
            D[i, j] = d
            D[j, i] = d

    return D



## Build the Outer Approximatio model
Introduce a scalar variable $\theta \in \mathbb{R}$ to represent the objective value
The Outer Approximation then replaces $\max g(x,t)$ by $\max\theta\quad \text{s.t.}\ \theta \le g(x,t)$, and then approximates $g$ by supporting hyperplances.

### Compute the gradient of $g(x,t)$
Since $f$ is quadratic,
$$
\nabla f(x) = Q\frac{x}{t}
$$
where $Q = [q_{ij}]$ is the distance matrix and $Q_{ii} = 0 \quad \forall i$

Now, 
$$
\nabla_xg(x,t) = \nabla f(\frac{x}{t}) = Q\frac{x}{t}
$$

Hence,
$$
\begin{aligned}
\frac{\partial g}{\partial t}
&= f\!\left(\frac{x}{t}\right)
  + t \left\langle \nabla f\!\left(\frac{x}{t}\right),
  \frac{\partial}{\partial t}\!\left(\frac{x}{t}\right) \right\rangle \\[6pt]
&= f\!\left(\frac{x}{t}\right)
  + t \left\langle \nabla f\!\left(\frac{x}{t}\right),
  \left(-\frac{x}{t^{2}}\right) \right\rangle \\[6pt]
&= f\!\left(\frac{x}{t}\right)
  - \left\langle \nabla f\!\left(\frac{x}{t}\right),
  \frac{x}{t} \right\rangle \\[6pt]
\frac{\partial g}{\partial t}(x,t)
= f\!\left(\frac{x}{t}\right)
- \left\langle Q\!\frac{x}{t}, \frac{x}{t} \right\rangle\\[6pt]
\end{aligned}
$$



### Supporting hyperplane
Let $(y,s)$ be a feasible point with $s>0$.

The supporting hyperplane of g at $(y,s)$ is
$$
h_{(y,s)}(x,t)
= g(y,s)
+ \langle \nabla_x g(y,s),\, x - y \rangle
+ \frac{\partial g}{\partial t}(y,s)\,(t - s).
$$
$$
\begin{aligned}
h_{(y,s)}(x,t)
&= g(y,s)
+ \left\langle Q\! \frac{y}{s},\, x - y \right\rangle \\[6pt]
&\quad + \left[
f\!\left(\frac{y}{s}\right)
- \left\langle Q\!\frac{y}{s},\, \frac{y}{s} \right\rangle
\right](t - s).
\end{aligned}
$$

Now, since $g$ is concave, $g(x,t) \le h_{(y,s)}(x,t) \quad \forall (x,t)$.


Therefore, our Outer Approximation model becomes
$$
\begin{aligned}
\max \quad & \theta \\[4pt]
\text{s.t.} \quad
& \sum_{i=1}^{m} x_i = k, \\[4pt]
& t = \sum_{i=1}^{m} w_i x_i, \\[4pt]
& x_i + x_j \le 1, \quad \forall (i,j): d_{ij} < \delta, \\[4pt]
& \sum_{i=1}^{m} a_i x_i \ge N, \\[4pt]
& \theta \le g(y,s)
+ \langle Q\frac{y}{s},\, x - y \rangle
+ \left[
f\!\left(\frac{y}{s}\right)
- \left\langle Q\!\frac{y}{s},\, \frac{y}{s} \right\rangle
\right](t - s),
\quad \forall (y,s) \in \mathcal{C}, \\[4pt]
& x_i \in \{0,1\}, \quad \theta \le \mathrm{UB}.
\end{aligned}
\qquad (\mathrm{OA})
$$


### Input data and setup model

In [4]:
# Set random seed for reproducibility
np.random.seed(21)

df = pd.read_csv("C:/Users/m/OneDrive - The University of Melbourne/Documents/Learning/2026 - AMSI Summer School/Mathematical Optimisation Theory and Applications/Silent-Disco-Glamping-Network/au-airports.csv")
print(df.head())
print(len(df))
df.isna().sum()


      id ident           type                                          name  \
0  27145  YSSY  large_airport  Sydney Kingsford Smith International Airport   
1  27066  YMML  large_airport                             Melbourne Airport   
2  26901  YBBN  large_airport                Brisbane International Airport   
3  27119  YPPH  large_airport                   Perth International Airport   
4  26904  YBCS  large_airport                  Cairns International Airport   

   latitude_deg  longitude_deg  elevation_ft continent country_name  \
0    -33.946098     151.177002          21.0        OC    Australia   
1    -37.670732     144.837898         434.0        OC    Australia   
2    -27.384199     153.117004          13.0        OC    Australia   
3    -31.940300     115.967003          67.0        OC    Australia   
4    -16.878921     145.749480          10.0        OC    Australia   

  iso_country  ... scheduled_service gps_code icao_code iata_code  local_code  \
0          AU  ..

id                      0
ident                   0
type                    0
name                    0
latitude_deg            0
longitude_deg           0
elevation_ft         1178
continent               0
country_name            0
iso_country             0
region_name             0
iso_region              0
local_region            0
municipality         1182
scheduled_service       0
gps_code              785
icao_code            2231
iata_code            2166
local_code           2446
home_link            2646
wikipedia_link       2275
keywords             2536
score                   0
last_updated            0
dtype: int64

In [5]:
df = df[df["type"].isin(["small_airport", "medium_airport"])] # only take small and medium airports to ensure remoteness 
n = len(df) #number of potential locations
print(f"Number of potential locations: {n}")
k = 10 #number of sites selected (It's logical that there is a small number of sites since this is for high-end tourism)
location = df[["latitude_deg", "longitude_deg"]].to_numpy() #Make location points in R^2
delta = 1000 #(km)minimum distance between selected sites (to ensure total exclusivity)

# get distance matrix
matrix = haversine_matrix(location)


# get upportbound for theta
upperbound = 1 / 2 * sum(matrix[i, j] for i in range(n) for j in range(n))

# get starting point
xk = np.zeros(n)
xk[:k] = 1

Number of potential locations: 2256


#### Create fixed cost data using the airstrips data
- Supply hubs: The supply hubs are the state capitals: Melbourne, Sydney, Hobart, .... The remoteness is computed by $d_i^h = \min_{h \in capitals} distance(i,h)$
- Fixed cost $w_i$
$$
w_i = \alpha d_i^h +\beta \text{size\_penalty}_i +\epsilon_i
$$
where size_penalty: is 1 for small airport and 0.5 for medium airport, $\epsilon$ is small random noise
- Capacity $a_i$: small airport can have 15 tents, medium airport can have 40 tents
- Target tents: $N = \gamma\frac{\sum_{i=1}^ma_i}{m/k}$

In [6]:
capitals = np.array([
    [-33.8688, 151.2093],  # Sydney
    [-37.8136, 144.9631],  # Melbourne
    [-27.4698, 153.0251],  # Brisbane
    [-31.9505, 115.8605],  # Perth
    [-34.9285, 138.6007],  # Adelaide
    [-42.8821, 147.3272],  # Hobart
    [-12.4634, 130.8456],  # Darwin
    [-35.2809, 149.1300],  # Canberra
])


# Distance from each airstrip to nearest hub
hub_dist = cdist(location, capitals).min(axis=1)

# Normalise distances
hub_dist = (hub_dist - hub_dist.min()) / (hub_dist.max() - hub_dist.min())

# Fixed cost w_i: combination of remoteness and infrastructure penalty
size_penalty = np.where(
    df["type"] == "small_airport", 1.0, 0.5
)

w = (
    5 * hub_dist              # remoteness cost
    + 3 * size_penalty        # infrastructure penalty
    + np.random.uniform(0, 1, n)  # noise
)
tk = np.sum(w *xk)  

# Capacity a_i
a = np.where(
    df["type"] == "small_airport",
    np.random.randint(5, 15, n),
    np.random.randint(20, 50, n)
)

# Target number of tents 
gamma = 0.6
N = int(gamma * np.sum(a) / (n / k))  # scalable, usually feasible

## Master Problem
$$
\begin{aligned}
\max \quad & \theta \\[4pt]
\text{s.t.} \quad
& \sum_{i=1}^{m} x_i = k, \\[4pt]
& t = \sum_{i=1}^{m} w_i x_i, \\[4pt]
& x_i + x_j \le 1, \quad \forall (i,j): d_{ij} < \delta, \\[4pt]
& \sum_{i=1}^{m} a_i x_i \ge N, \\[4pt]
& x_i \in \{0,1\}, \quad \theta \le \mathrm{UB}.
\end{aligned}
\qquad
$$

In [7]:
# setup model
m = Model("OuterApprox")

# set variables
x = m.addVars(n, vtype=GRB.BINARY, name="x")

# add cost variable t
t = m.addVar(name="t", vtype=GRB.CONTINUOUS)

#add cost definition constraint
m.addConstr(
    t == gp.quicksum(w[i] * x[i] for i in range(n)),
    name="cost_def"
)

# add capacity constraint
m.addConstr(
    gp.quicksum(a[i] * x[i] for i in range(n)) >= N,
    name="capacity"
)

# add distance constraints
for i in range(n):
    for j in range(i + 1, n):
        if matrix[i, j] < delta:
            m.addConstr(x[i] + x[j] <= 1)

# set theta
theta = m.addVar(name="theta", ub=upperbound, vtype=GRB.CONTINUOUS)

# add constraints: cardinality constraint
m.addConstr(gp.quicksum(x[i] for i in range(n)) == k)

# objective function
m.setObjective(theta, GRB.MAXIMIZE)

Set parameter Username
Set parameter LicenseID to value 2767716
Academic license - for non-commercial use only - expires 2027-01-20


## Calculate objective function, and gradient

$$
f(x) = \tfrac{1}{2} \langle Qx,x\rangle
$$
and
$$
\nabla f(x) = Qx.
$$

In [8]:
def f(x: np.ndarray, matrix: np.ndarray) -> float:
    """
    Calculates the value of the quadratic objective function.

    The function computes 0.5 * x' * Q * x, where Q is the distance matrix.

    Args:
        x: A numpy array representing the solution vector.
        matrix: The symmetric distance matrix (Q).

    Returns:
        The calculated value of the objective function as a float.
    """
    return x.dot(matrix).dot(x) / 2

def g(x,t,Q):
    """
    Calculates the value of objective function g(t,x).

    g(x,t) = t * f(x/t)

    Args:
        x: A numpy array representing the solution vector.
        t: A positive scalar (total cost).
        Q: The symmetric distance matrix.

    Returns:
        The value of the perspective objective function.
    """
    z = x/t
    return t * f(z, Q)

def dg_dx(x: np.ndarray, t: float, Q: np.ndarray) -> np.ndarray:
    """
    Calculates the gradient of g(x,t) with respect to x.

    nabla_x g(x,t) = Q * (x / t)

    Args:
        x: A numpy array representing the solution vector.
        t: A positive scalar.
        Q: The symmetric distance matrix.

    Returns:
        A numpy array representing the gradient with respect to x.
    """
    return Q @ (x / t)

def dg_dt(x: np.ndarray, t: float, Q: np.ndarray) -> float:
    """
    Calculates the derivative of g(x,t) with respect to t.

    partial_g/partial-t = f(x/t) - <Q(x/t), x/t>

    Args:
        x: A numpy array representing the solution vector.
        t: A positive scalar.
        Q: The symmetric distance matrix.

    Returns:
        The derivative of g with respect to t.
    """
    z = x / t
    return f(z, Q) - (Q @ z) @ z




## Define Cutting plane
$$
\begin{aligned}
\theta
&\le g(y,s)
+ \left\langle Q\!\left(\frac{y}{s}\right),\, x - y \right\rangle
+ \left[
f\!\left(\frac{y}{s}\right)
- \left\langle Q\!\left(\frac{y}{s}\right),\, \frac{y}{s} \right\rangle
\right](t - s)
= g(y,s)
+ \left\langle Q\!\left(\frac{y}{s}\right),\, x - y \right\rangle
- f\!\left(\frac{y}{s}\right)(t - s).
\end{aligned}
$$
$\textbf{Stopping condition}$: upperbound $=$ lowerbound


lowerbound $= g(x^k,t^k) = t^k f(\frac{x^k}{t^k}), t^k = \sum_{i=1}^mw_ix_i^k$

upperbound $= \theta^k$





In [9]:
# get lower bound
lowerbound = g(xk, tk, matrix)

# set up first step
num_iter = 0
m.setParam("OutputFlag", 0)

while upperbound - lowerbound > 0.01 and num_iter < 100: # continue if the upperbound is larger than lowerbound by 0.01

    # previous solution (y, s)
    y = xk.copy()
    s = tk

    # compute objective + gradients
    g_val = g(y, s, matrix)                 
    grad_x = dg_dx(y, s, matrix)            
    grad_t = dg_dt(y, s, matrix)            

    # update the cutting plane
    m.addConstr(
        theta
        <= g_val
        + gp.quicksum(grad_x[i] * (x[i] - y[i]) for i in range(n))
        + grad_t * (t - s)
    )

    # solve the new model
    m.optimize()

    if m.status != GRB.OPTIMAL:
        print("No optimal solution found. Status =", m.status)
        break

    # get new vector
    xk = np.array([x[i].X for i in range(n)])
    tk = np.sum(w * xk)

    # update lower bounds
    lowerbound = g(xk, tk, matrix)

    # update upper bounds
    upperbound = m.ObjVal            
    # update steps
    num_iter += 1

print(colored("Number of iteration:", "green", attrs=["bold"]), num_iter)
print(colored("Objective value:", "green", attrs=["bold"]), m.ObjVal)
print(colored("Solution:", "green", attrs=["bold"]), [x[i].X for i in range(n)])



Number of iteration: 6
Objective value: 2992.1271483921273
Solution: [0.0, 0.0, -0.0, 0.0, 0.0, 1.0, 0.0, 0.0, -0.0, 0.0, 0.0, -0.0, 0.0, -0.0, 0.0, 0.0, -0.0, -0.0, 0.0, 1.0, 0.0, 0.0, -0.0, 0.0, 0.0, 0.0, 0.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0, -0.0, -0.0, 0.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0, 0.0, 0.0, 0.0, -0.0, 1.0, 0.0, 0.0, 0.0, 0.0, -0.0, 0.0, -0.0, 0.0, -0.0, 0.0, -0.0, 0.0, -0.0, -0.0, -0.0, -0.0, 0.0, 1.0, -0.0, -0.0, -0.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0, 0.0, 0.0, 0.0, 0.0, -0.0, -0.0, -0.0, 0.0, 0.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0, 0.0, -0.0, -0.0, 0.0, -0.0, -0.0, 1.0, 0.0, 1.0, -0.0, -0.0, 0.0, 0.0, 0.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0, 0.0, 0.0, 0.0, -0.0, -0.0, 0.0, 0.0, 0.0, -0.0, -0.0, 0.0, 0.0, 0.0, -0.0, 0.0, 0.0, 0.0, 0.0, -0.0, 1.0, -0.0, 0.0, 0.0, -0.0, 0.0, 0.0, 1.

In [10]:
def outer_approximation_glamping(
    matrix: np.ndarray,
    w: np.ndarray,
    a: np.ndarray,
    delta: float,
    k: int,
    N: int,
    timeout: float = 3600,
    verbose: bool = True,
):
    """
    Solves the Silent Disco Glamping Network problem using Outer Approximation algorithm.

    This function iteratively builds a master problem with cutting planes
    derived from the perspective function g(x,t) and its gradients. 
    It continues until the optimality gap is within a specified tolerance or a 
    maximum number of iterations is reached.

    Args:
        matrix: Haversine distance matrix (Q) in km.
        w: Fixed cost vector.
        a: Capacity vector.
        delta: Minimum distance between selected sites (km).
        k: Number of sites to select.
        N: Minimum total capacity.
        timeout: Solver time limit (seconds).
        verbose: Print iteration progress.

    Returns:
        (final_obj, runtime, solution_vector, optimality_gap)
    """

    n = len(w)
    m = None

    try:
        # Upper bound for theta
        upperbound = 0.5 * np.sum(matrix)

        # Build master problem
   
        m = Model("OuterApprox_Glamping")
        m.setParam("TimeLimit", timeout)
        m.setParam("OutputFlag", 0)

        x = m.addVars(n, vtype=GRB.BINARY, name="x")
        t = m.addVar(vtype=GRB.CONTINUOUS, name="t")
        theta = m.addVar(vtype=GRB.CONTINUOUS, ub=upperbound, name="theta")

        # Cardinality
        m.addConstr(gp.quicksum(x[i] for i in range(n)) == k)

        # Cost definition
        m.addConstr(t == gp.quicksum(w[i] * x[i] for i in range(n)))

        # Capacity constraint
        m.addConstr(gp.quicksum(a[i] * x[i] for i in range(n)) >= N)

        # Distance exclusion
        for i in range(n):
            for j in range(i + 1, n):
                if matrix[i, j] < delta:
                    m.addConstr(x[i] + x[j] <= 1)

        # Objective
        m.setObjective(theta, GRB.MAXIMIZE)

        # Initial master solve 
        m.optimize()
        if m.status != GRB.OPTIMAL:
            raise RuntimeError("Initial master problem infeasible.")

        xk = np.array([x[i].X for i in range(n)])
        tk = np.sum(w * xk)

        lowerbound = g(xk, tk, matrix)
        upperbound = m.ObjVal

        num_iter = 0
        time_solve = m.Runtime

        # Outer Approximation loop
        while upperbound - lowerbound > 1e-2 and num_iter < 100:

            y = xk.copy()
            s = tk

            g_val = g(y, s, matrix)
            grad_x = dg_dx(y, s, matrix)
            grad_t = dg_dt(y, s, matrix)

            # Add cutting plane
            m.addConstr(
                theta
                <= g_val
                + gp.quicksum(grad_x[i] * (x[i] - y[i]) for i in range(n))
                + grad_t * (t - s)
            )

            # Solve master
            m.optimize()
            if m.status != GRB.OPTIMAL:
                if verbose:
                    print("Master became infeasible at iteration", num_iter)
                break

            xk = np.array([x[i].X for i in range(n)])
            tk = np.sum(w * xk)

            lowerbound = g(xk, tk, matrix)
            upperbound = m.ObjVal

            num_iter += 1
            time_solve += m.Runtime

            if verbose:
                print(
                    f"Iteration {num_iter}: "
                    f"UB = {upperbound:.4f}, "
                    f"LB = {lowerbound:.4f}"
                )

        gap = (
            (upperbound - lowerbound) / upperbound * 100
            if upperbound > 0 else 0
        )

        return (
            upperbound,
            time_solve,
            [x[i].X for i in range(n)],
            gap,
        )

    finally:
        if m is not None:
            m.dispose()


In [11]:
obj, runtime, sol, gap = outer_approximation_glamping(
    matrix=matrix,
    w=w,
    a=a,
    delta=delta,
    k=k,
    N=N,
    verbose=True
)


Set parameter TimeLimit to value 3600
Iteration 1: UB = 2654.7255, LB = 2845.7696


### Visualise the result

In [12]:
df_plot = df.copy().reset_index(drop=True)
df_plot["selected"] = sol

#Check how many sites were selected
df_plot["selected"].sum()




np.float64(10.0)

In [13]:
from wcwidth import width


df_plot["selected_cat"] = df_plot["selected"].map({0: "Not Selected", 1: "Selected"})

fig = px.scatter_geo(
    df_plot,
    lat="latitude_deg",
    lon="longitude_deg",
    color="selected_cat",
    color_discrete_map={"Not Selected": "royalblue", "Selected": "red"},
    size=df_plot["selected"].map({1:16, 0:6}),   #.apply(lambda x: 12 if x == 1 else 4),
    hover_name="name",
    title="Optimal locations for Silent Disco Glamping Network",
)

fig.update_layout(
    width = 1000,
    height = 700,
    geo=dict(
        scope="oceania",
        projection_type="mercator",
        showland=True,
        landcolor="rgb(230, 230, 230)",
        lataxis_range=[-45, -10],   # zoom to Australia
        lonaxis_range=[110, 155],
    ),
    coloraxis_showscale=False,
    legend_title_text="Legend",
)
fig.show()


### Find the maximum $\delta$ so that the model is still feasible

In [14]:
def is_feasible_delta(
    matrix: np.ndarray,
    a: np.ndarray,
    w: np.ndarray,
    k: int,
    N: int,
    delta: float,
    time_limit: float = 30,
) -> bool:
    """ 
    Check whether the Silent Disco Glamping Network problem is feasible for a given minimum distance threshold delta 
    by determining if a selection of k locations satisfying all constraints for a given value of k.
    
    This function solves a feasibility version of the problem without optimising the original objective

    Args:
        matrix: Haversine distance matrix (Q) in km.
        a: Capacity vector.
        w: Fixed cost vector.
        k: Number of sites to select.
        N: Minimum total capacity.
        delta: Minimum distance between selected sites (km).
        time_limit: Solver time limit in seconds.

    Returns:
        - True if there exists a feasible solution that satisfies the cardinality, capacity, and distance constraints for the given delta.
        - False otherwise.
    """
    n = len(a)

    m = Model("FeasibilityCheck")
    m.setParam("OutputFlag", 0)
    m.setParam("TimeLimit", time_limit)

    x = m.addVars(n, vtype=GRB.BINARY, name="x")

    # Cardinality
    m.addConstr(gp.quicksum(x[i] for i in range(n)) == k)

    # Capacity
    m.addConstr(gp.quicksum(a[i] * x[i] for i in range(n)) >= N)

    # Distance exclusion
    for i in range(n):
        for j in range(i + 1, n):
            if matrix[i, j] < delta:
                m.addConstr(x[i] + x[j] <= 1)

    # Dummy objective
    m.setObjective(0, GRB.MAXIMIZE)

    m.optimize()

    feasible = m.status in (GRB.OPTIMAL, GRB.SUBOPTIMAL)
    m.dispose()

    return feasible


In [15]:
def find_max_delta(
    matrix: np.ndarray,
    a: np.ndarray,
    w: np.ndarray,
    k: int,
    N: int,
    tol: float = 1.0,   # km tolerance
) -> float:
    """
    Computes the maximum feasible minimum-distance threshold delta
    
    This function performs a binary search over delta to determine the largest value
    for which the Silent Disco Glamping Network problem is feasible.
    
    Args:
        matrix: Haversine distance matrix (Q) in km.
        a: Capacity vector.
        w: Fixed cost vector.
        k: Number of sites to select.
        N: Minimum total capacity.
        tol: Tolerance for binary search convergence (in km).
        
    Returns:
        The maximum feasible delta value (in km) that the model admits at least one
        feasible selection of k locations.
    """
    delta_low = 0.0
    delta_high = np.max(matrix)

    best_delta = 0.0

    while delta_high - delta_low > tol:
        delta_mid = 0.5 * (delta_low + delta_high)

        if is_feasible_delta(matrix, a, w, k, N, delta_mid):
            best_delta = delta_mid
            delta_low = delta_mid   # try larger delta
        else:
            delta_high = delta_mid  # too large, reduce

    return best_delta


In [16]:
# To appy the function and find the maximum delta
delta_star = find_max_delta(
    matrix=matrix,
    a=a,
    w=w,
    k=k,
    N=N,
    tol=10.0   # ±10 km precision
)

print(f"Maximum feasible delta ≈ {delta_star:.1f} km")


Maximum feasible delta ≈ 1321.1 km


In [17]:
# To check whether our obtained delta_star is the maximum feasible by testing a slightly larger value
is_feasible_delta(matrix, a, w, k, N, delta_star + 50)

False